In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Read the parquet file
table = pq.read_table('./websites.snappy.parquet')
df = table.to_pandas()

# Extract the list of websites
websites = df['domain'].tolist()

# Iterate through each website
for website in websites:

    if not website.startswith('http'):
        website = 'http://' + website

    try:
        # Make a request to the website
        response = requests.get(website)

        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        address_elements = soup.find_all(['span', 'p', 'a'])

        for element in address_elements:
            address_text = element.text

            postcode = re.search(r'\b\d{5}\b', address_text)

            if postcode:
                element_text = " ".join(element.stripped_strings)
                print(element_text)
                break

    except Exception as e:
        print(f"Error accessing website: {e}")
